In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from tqdm import tqdm

In [9]:
# 데이터 로드
path = "../stat_proj/data/prepro_data_ver.2.3_js.csv"
df = pd.read_csv(path)

#df = pd.read_csv('preprocessed_data_ver.2.3._IRR+LOG+BINARY.csv')

In [10]:
# 모든 열 다 보기
pd.set_option('display.max_columns', None)

# 모든 행 다 보기
pd.set_option('display.max_rows', None)

# 열 너비 넉넉하게
pd.set_option('display.width', None)

# 열 안의 값 자르지 않게
pd.set_option('display.max_colwidth', None)

In [11]:
df.head()

,funded_amnt,term,installment,emp_length,annual_inc,issue_d,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,recoveries,collection_recovery_fee,last_pymnt_d,last_fico_range_high,collections_12_mths_ex_med,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,default,addr_state_AK,addr_state_AL,addr_state_AR,addr_state_AZ,addr_state_CA,addr_state_CO,addr_state_CT,addr_state_DC,addr_state_DE,addr_state_FL,addr_state_GA,addr_state_HI,addr_state_ID,addr_state_IL,addr_state_IN,addr_state_KS,addr_state_KY,addr_state_LA,addr_state_MA,addr_state_MD,addr_state_ME,addr_state_MI,addr_state_MN,addr_state_MO,addr_state_MS,addr_state_MT,addr_state_NC,addr_state_ND,addr_state_NE,addr_state_NH,addr_state_NJ,addr_state_NM,addr_state_NV,addr_state_NY,addr_state_OH,addr_state_OK,addr_state_OR,addr_state_PA,addr_state_RI,addr_state_SC,addr_state_SD,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY,addr_state_nan,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,home_ownership_nan,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,purpose_nan,emp_length_missing,percent_bc_gt_75_missing,delinq_amnt_missing,tax_liens_missing,mths_since_last_delinq_missing,fico_range_low_missing,open_il_12m_missing,fico_range_high_missing,mths_since_recent_inq_missing,open_acc_missing,all_util_missing,open_acc_6m_missing,annual_inc_missing,inq_last_6mths_missing,collection_recovery_fee_missing,mths_since_recent_revol_delinq_missing,last_fico_range_high_missing,acc_now_delinq_missing,open_rv_24m_missing,total_bal_il_missing,total_acc_missing,revol_bal_missing,mths_since_rcnt_il_missing,open_act_il_missing,collections_12_mths_ex_med_missing,pub_rec_bankruptcies_missing,pub_rec_missing,funded_amnt_missing,open_rv_12m_missing,open_il_24m_missing,delinq_2yrs_missing,inq_last_12m_missing,chargeoff_within_12_mths_missing,il_util_missing,installment_missing,max_bal_bc_missing,recoveries_missing
0,4425.0,36,160.53,10.0,11.156265,2016-03-01,2.776954,1,665.0,669.0,1.0,8.0,14.0,0,8.217439,63.4,21.0,0.00,0.00,Mar-2019,569.0,0,0,0,61209.0,2.0,9.0,0.0,0.0,29.0,57505.0,102.0,2.0,3.0,7.565793,100.0,8.476580,2.0,3.0,8.383205,4.584967,94.6,0,0.0,144.0,124.0,2.0,2.0,0.0,13.0,4.0,37.0,3.0,5.0,2.0,2.0,16.0,5.0,5.0,5.0,14.0,0,0,1,2.0,73.7,100.0,0,0,11.021069,11.022066,7.824446,10.939320,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,4000.0,36,136.89,0.0,10.596660,2012-08-01,2.598979,0,705.0,709.0,3.0,0.0,4.0,0,9.015420,36.6,9.0,200.57,1.76,Feb-2013,514.0,0,0,0,79938.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,10.128669,0.0,5.0,8.902932,9.392745,0.0,0,0.0,0.0,162.0,8.0,6.0,0.0,11.0,0.0,0.0,0.0,5.0,2.0,7.0,7.0,7.0,13.0,5.0,4.0,0,0,0,2.0,100.0,0.0,0,0,11.647898,9.015420,9.392745,10.372866,1,0,0,0,0,1,0,0,0,0,0,0,0,0,

In [15]:
# 독립변수에서 제외할 변수 목록
drop_cols = [
    'term', 'last_pymnt_d', 'installment', 'funded_amnt',
    'recoveries', 'collection_recovery_fee', 'default', 'issue_d'
]

X = df.drop(columns=drop_cols)
y = df['default']

In [17]:
from fredapi import Fred

# FRED API 연결
fred = Fred(api_key="f9ef939a9a1eeff85d31ed2c984507d9")  # 🔑 실제 API 키로 대체해야 함

# 미국 3년 만기 국채 수익률(GS3) 시계열 불러오기
series = fred.get_series('GS3')  # pandas.Series 형식으로 반환됨


print(series.head())

1953-04-01    2.51
1953-05-01    2.72
1953-06-01    2.74
1953-07-01    2.62
1953-08-01    2.58
dtype: float64


In [19]:
series.shape

(868,)

In [20]:
def get_nearest_rate(issue_date, rate_series):
    if pd.isnull(issue_date):
        return np.nan
    try:
        idx = rate_series.index.get_indexer([issue_date], method='nearest')[0]
        return rate_series.iloc[idx] / 100  # % 단위 → 소수로 변환
    except Exception as e:
        print(f"Error: {issue_date} ▶ {e}")
        return np.nan


df['risk_free_rate'] = df['issue_d'].apply(lambda x: get_nearest_rate(x, series))

In [21]:
df['last_pymnt_d'] = pd.to_datetime(df['last_pymnt_d'])
df['issue_d'] = pd.to_datetime(df['issue_d'])  # 혹시 모르니 이것도

/var/folders/2q/_8m2fvxd4pv04q2b3gk8j0bm0000gn/T/ipykernel_59799/4242469769.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['last_pymnt_d'] = pd.to_datetime(df['last_pymnt_d'])


In [22]:
print(df[['issue_d', 'risk_free_rate', 'last_pymnt_d']].head())

     issue_d  risk_free_rate last_pymnt_d
0 2016-03-01          0.0104   2019-03-01
1 2012-08-01          0.0037   2013-02-01
2 2019-12-01          0.0163   2020-05-01
3 2014-03-01          0.0082   2015-11-01
4 2018-08-01          0.0271   2019-07-01


In [23]:
from dateutil.relativedelta import relativedelta

def create_cash_flow_from_dates(row):
    try:
        term = int(row['term'])
        default = int(row['default'])
        issue_d = row['issue_d']  # 이미 datetime 형식
        last_pymnt_d = pd.to_datetime(row['last_pymnt_d'])  # 이건 문자열일 수도 있으니 변환
        installment = float(row['installment'])
        funded_amnt = float(row['funded_amnt'])
        recoveries = float(row['recoveries'])
        collection_fee = float(row['collection_recovery_fee'])

        # 첫 현금흐름: 대출 실행
        cash_flow = [-funded_amnt]

        if pd.isnull(issue_d) or pd.isnull(last_pymnt_d):
            return np.nan

        # 몇 회차까지 납입했는지 계산
        delta = relativedelta(last_pymnt_d, issue_d)
        last_pymnt_num = delta.years * 12 + delta.months

        for month in range(1, term + 1):
            if default == 1:
                if month <= last_pymnt_num:
                    cash_flow.append(installment)
                elif month == last_pymnt_num + 1:
                    cash_flow.append(recoveries - collection_fee)
                else:
                    cash_flow.append(0)
            else:
                cash_flow.append(installment)

        return cash_flow

    except Exception as e:
        print(f"[오류] index={row.name}, error={e}")
        return np.nan


In [24]:
df['cash_flow'] = df.apply(create_cash_flow_from_dates, axis=1)

In [25]:
!pip install numpy-financial


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [26]:
import numpy_financial as npf

def get_irr(cash_flow):
    if not isinstance(cash_flow, list) or len(cash_flow) == 0:
        return np.nan
    irr_monthly = npf.irr(cash_flow)
    if irr_monthly is None or np.isnan(irr_monthly):
        return np.nan
    return (1 + irr_monthly) ** 12 - 1  # 연환산

In [27]:
df['irr'] = df['cash_flow'].apply(get_irr)

In [28]:
df['irr'] = df['irr'].fillna(df['risk_free_rate'])

df['irr'] = df['cash_flow'].apply(lambda x: np.irr(x) if isinstance(x, list) else np.nan)




---



In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [30]:
# 결과 저장용 리스트
best_thresholds = []
validation_sharpes = []
test_sharpes = []
test_approval_rates = []
test_irr_means = []
test_irr_positive_rates = []


# Sharpe 계산 함수
def calculate_sharpe(returns, risk_free_rates):
    excess = returns - risk_free_rates
    if excess.std(ddof=1) == 0:
        return -np.inf
    return excess.mean() / excess.std(ddof=1)

In [34]:
for i in range(100):
    # Train-test-split
    # 먼저 test 데이터 분리 (20%)
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y, test_size=0.2, random_state=i, stratify=y
    )

    # 남은 80% 중에서 train (60%) / val (20%) 분리
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=0.25, random_state=i, stratify=y_temp
    )
    # 랜덤 포레스트 모델 학습 (기본 하이퍼파라미터)
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)

    # 검증 세트에 대한 부도 확률 예측
    y_pred_proba = model.predict_proba(X_val)[:, 1]

    # 사용할 threshold 후보들
    thresholds = np.arange(0.0, 0.95, 0.05)

    best_sharpe = -np.inf
    best_threshold = None

    # X_val의 인덱스를 이용해 df에서 대응하는 행 추출
    val_indices = X_val.index
    df_val = df.loc[val_indices]

    for threshold in thresholds:
        approved_mask = y_pred_proba <= threshold
        denied_mask = ~approved_mask

        selected = df_val.copy()
        
        # 승인된 건: 실제 IRR 사용
        selected.loc[approved_mask, 'irr_adj'] = selected.loc[approved_mask, 'irr']

        # 거절된 건: IRR을 국채수익률로 설정
        selected.loc[denied_mask, 'irr_adj'] = selected.loc[denied_mask, 'risk_free_rate']

        # 수익률과 무위험수익률 추출
        returns = selected['irr_adj']
        risk_free = selected['risk_free_rate']

        # NaN 제거
        valid = returns.notnull() & risk_free.notnull()
        if valid.sum() < 2:
            continue

        sharpe = calculate_sharpe(returns[valid], risk_free[valid])

        if sharpe > best_sharpe:
            best_sharpe = sharpe
            best_threshold = threshold

    # 저장
    best_thresholds.append(best_threshold)
    validation_sharpes.append(best_sharpe)

    # 4. Test 성능 측정
    y_test_proba = model.predict_proba(X_test)[:, 1]
    test_approved_mask = y_test_proba <= best_threshold
    df_test = df.loc[X_test.index]
    test_selected = df_test[test_approved_mask]

    returns_test = test_selected['irr']
    risk_free_test = test_selected['risk_free_rate']

    valid = returns_test.notnull() & risk_free_test.notnull()
    returns_test = returns_test[valid]
    risk_free_test = risk_free_test[valid]

    sharpe_test = calculate_sharpe(returns_test, risk_free_test)

    test_sharpes.append(sharpe_test)
    test_approval_rates.append(len(returns_test) / len(df_test))
    test_irr_means.append(returns_test.mean())
    test_irr_positive_rates.append((returns_test > 0).mean())

In [36]:
print("Best thresholds from 2 runs:")
print(best_thresholds)

print("\nValidation Sharpe Ratios from 2 runs:")
print(validation_sharpes)

print("\nTest Sharpe Ratios from 2 runs:")
print(test_sharpes)

print("\nTest Approval Rates:")
print(test_approval_rates)

print("\nTest IRR Means:")
print(test_irr_means)

print("\nTest IRR Positive Rates:")
print(test_irr_positive_rates)


Best thresholds from 2 runs:
[np.float64(0.15000000000000002), np.float64(0.1), np.float64(0.15000000000000002), np.float64(0.1), np.float64(0.15000000000000002), np.float64(0.1), np.float64(0.2), np.float64(0.15000000000000002), np.float64(0.15000000000000002), np.float64(0.25), np.float64(0.2), np.float64(0.2), np.float64(0.15000000000000002), np.float64(0.1), np.float64(0.15000000000000002), np.float64(0.1), np.float64(0.2), np.float64(0.15000000000000002), np.float64(0.15000000000000002), np.float64(0.25), np.float64(0.2), np.float64(0.2), np.float64(0.15000000000000002), np.float64(0.15000000000000002), np.float64(0.2), np.float64(0.1), np.float64(0.15000000000000002), np.float64(0.15000000000000002), np.float64(0.15000000000000002), np.float64(0.1), np.float64(0.1), np.float64(0.15000000000000002), np.float64(0.15000000000000002), np.float64(0.15000000000000002), np.float64(0.15000000000000002), np.float64(0.2), np.float64(0.1), np.float64(0.2), np.float64(0.15000000000000002), n